# Совместная работа по машинному обучения

### Группа выполнившая работу
* ##### 22П-1

### Состав команды
* ##### Рахмаев Данил
* ##### Петрова Яна

## Подключение библиотек

In [12]:
import pandas
import pandas as pd
import nltk
import string
import re
import pymorphy3

import warnings
warnings.filterwarnings('ignore')

## Подключение датасета

In [14]:
df = pd.read_csv("real_estate_data.csv")

In [15]:
df.head()

,id,type,sub_type,start_date,end_date,listing_type,tom,building_age,total_floor_count,floor_no,room_count,size,address,furnished,heating_type,price,price_currency
0,1,Konut,Rezidans,12/10/18,1/9/19,2,30,0,20 ve üzeri,2,2+1,90.0,İstanbul/Kartal/Kordonboyu,NaN,Fancoil,3500.0,TRY
1,2,Konut,Daire,2/13/19,NaN,1,14,0,20 ve üzeri,20 ve üzeri,1+0,43.0,İstanbul/Kartal/Kordonboyu,NaN,Fancoil,490000.0,TRY
2,3,Konut,Daire,10/9/18,11/8/18,1,30,0,1,Yüksek Giriş,2+1,NaN,Tekirdağ/Çorlu/Reşadiye,NaN,Fancoil,155000.0,TRY
3,4,Konut,Rezidans,9/10/18,10/10/18,1,30,3,20 ve üzeri,20 ve üzeri,6+1,450.0,İstanbul/Beşiktaş/Levent,NaN,Fancoil,32500000.0,TRY
4,5,Konut,Rezidans,12/10/18,1/9/19,1,30,0,20 ve üzeri,2,2+1,90.0,İstanbul/Kartal/Kordonboyu,NaN,Fancoil,1450000.0,TRY


## Описание данных

* id - Уникальный идентификационный номер объявления о недвижимости.
* type - Основной тип объекта недвижимости.
* sub_type - Подтип объекта недвижимости, уточняющий категорию.
* start_date - Дата активации и размещения объявления на рынке.
* end_date - Дата снятия объявления с рынка.
* listing_type - Тип списка: на продажу ("Satılık") или в аренду ("Kiralık").
* tom - Время, в течение которого объявление было активно на рынке.
* building_age - Возраст здания, в котором находится объект.
* total_floor_count - Общее количество этажей в здании.
* floor_no - Этаж, на котором расположен объект, включая специальные категории (например, "Цокольный", "Пентхаус").
* room_count - Количество комнат в формате "X+Y".
* size - Площадь объекта в квадратных метрах.
* address - Адрес объекта.
* furnished - Уровень меблировки объекта.
* heating_type - Тип системы отопления в здании.
* price - Цена объекта недвижимости.
* price_currency - Валюта, в которой указана цена.

#### Целевой признак - Price (Цена объекта недвижимости).

## Разведочный анализ данных

In [31]:
df.shape #Количества строк и признаков

(403487, 17)

* В датасете присутствует 403487 записи, и 17 полей для записей

In [36]:
df.dtypes #Признаки и их тип

id                     int64
type                  object
sub_type              object
start_date            object
end_date              object
listing_type           int64
tom                    int64
building_age          object
total_floor_count     object
floor_no              object
room_count            object
size                 float64
address               object
furnished            float64
heating_type          object
price                float64
price_currency        object
dtype: object

* В датасете приобладает строковый тип данных

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403487 entries, 0 to 403486
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 403487 non-null  int64  
 1   type               403487 non-null  object 
 2   sub_type           403487 non-null  object 
 3   start_date         403487 non-null  object 
 4   end_date           266298 non-null  object 
 5   listing_type       403487 non-null  int64  
 6   tom                403487 non-null  int64  
 7   building_age       376097 non-null  object 
 8   total_floor_count  375466 non-null  object 
 9   floor_no           368191 non-null  object 
 10  room_count         403487 non-null  object 
 11  size               257481 non-null  float64
 12  address            403487 non-null  object 
 13  furnished          0 non-null       float64
 14  heating_type       375517 non-null  object 
 15  price              402772 non-null  float64
 16  pr

* При анализе общений информации, можно заметить, что в датасете присутствуют пропуски в полях

In [55]:
# Выявление признаков с пропусками
for col in df.columns:
    print('Кол-во пропусков в колонке %s = %d' % (col, df[col].isna().sum()))

Кол-во пропусков в колонке id = 0
Кол-во пропусков в колонке type = 0
Кол-во пропусков в колонке sub_type = 0
Кол-во пропусков в колонке start_date = 0
Кол-во пропусков в колонке end_date = 137189
Кол-во пропусков в колонке listing_type = 0
Кол-во пропусков в колонке tom = 0
Кол-во пропусков в колонке building_age = 27390
Кол-во пропусков в колонке total_floor_count = 28021
Кол-во пропусков в колонке floor_no = 35296
Кол-во пропусков в колонке room_count = 0
Кол-во пропусков в колонке size = 146006
Кол-во пропусков в колонке address = 0
Кол-во пропусков в колонке furnished = 403487
Кол-во пропусков в колонке heating_type = 27970
Кол-во пропусков в колонке price = 715
Кол-во пропусков в колонке price_currency = 715


* Анализ показал что пропуски есть в половине признаков

In [57]:
# Статистика для всех (числовых) признаков
df.describe()

,id,listing_type,tom,size,furnished,price
count,403487.00000,403487.000000,403487.000000,257481.000000,0.0,4.027720e+05
mean,201744.00000,1.294235,57.022739,279.349094,NaN,3.546417e+05
std,116476.80837,0.467733,44.358933,9429.195331,NaN,4.809503e+06
min,1.00000,1.000000,0.000000,1.000000,NaN,-2.500000e+02
25%,100872.50000,1.000000,29.000000,85.000000,NaN,2.500000e+03
50%,201744.00000,1.000000,40.000000,110.000000,NaN,1.990000e+05
75%,302615.50000,2.000000,90.000000,140.000000,NaN,3.420000e+05
max,403487.00000,3.000000,180.000000,948235.000000,NaN,2.000000e+09


## Предварительная обработка данных